In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

In [2]:
url = "https://www.basketball-reference.com/leagues/NBA_2024_per_poss.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find_all('table')

In [3]:
df = pd.read_html(str(table))[0]
df = df[df.G != "G"].copy()
df.G = pd.to_numeric(df.G)
df = df[(~df.Player.duplicated()) & (df.G >= 58)].copy()
df.Pos = df.Pos.apply(lambda x: x.split("-")[0])

In [4]:
stats = ["PTS", "TRB", "AST", "STL", "BLK", "FG%", "3P%", "FT%"]
positions = df.Pos.unique()
positions_df = []

In [5]:
for position in positions:
    ndf = df[df.Pos == position].copy()
    ndf["TOT_VAR"] = 0
    stats_var = []
    for stat in stats:
        ndf[stat] = pd.to_numeric(ndf[stat])
        med = ndf[stat].median()
        var = ndf[stat].std()
        var_name = f"{stat}_VAR"
        ndf[var_name] = round((ndf[stat] - med)/var, 2)
        ndf[var_name] = ndf[var_name].fillna(3) #Assume variance of 3 
        stats_var.append(var_name)
        ndf["TOT_VAR"] = round(ndf["TOT_VAR"] + abs(ndf[var_name]), 2)
        
    ndf = ndf[["Player", "Tm", "G", "Pos"] + stats + stats_var + ["TOT_VAR"]].copy()
    positions_df.append(ndf)

In [6]:
df = pd.concat(positions_df)
df.sort_values(by="Player").reset_index(drop=True).to_csv('variances_2024.csv')